In [1]:
from StoredQueries import POS_transactions
import pandas as pd

In [2]:
POS_df = POS_transactions()

In [3]:
# Create list of all transactions grouped by each transaction
POS_df.head()

,PartRno,PartId,PartName,LineItemID,TransactionID,Seq,PartRno,PartPriceRno,Qty,PartMeasureQty,UnitPrice,ExtPrice,CreDtTm
0,202,P70831298,TAIL GATE,3,2,1,202,202.0,1,1.000,29.9900,29.9900,2021-03-19 09:51:01.643
1,122,I70831252,SEAT BUCKET MANUAL,4,3,1,122,122.0,2,1.000,15.9900,31.9800,2021-03-19 10:02:09.660
2,75,E70831154,COIL,5,4,1,75,75.0,1,1.000,4.9900,4.9900,2021-03-19 10:07:59.860
3,125,I70831255,SEAT BELT (EACH),6,5,1,125,125.0,2,1.000,5.9900,11.9800,2021-03-19 11:04:12.900
4,10,A70831176,CAM SHAFT,7,6,1,10,10.0,1,1.000,15.9900,15.9900,2021-03-19 11:05:31.170


In [4]:
# Grouping the data by 'TransactionID' and aggregating the 'PartName' into a list
grouped_df = POS_df.groupby('TransactionID')['PartName'].apply(list).reset_index()

# Renaming the columns
grouped_df.columns = ['TransactionID', 'PartNames']



In [5]:
grouped_df.head(20)

,TransactionID,PartNames
0,2,[TAIL GATE]
1,3,[SEAT BUCKET MANUAL]
2,4,[COIL]
3,5,[SEAT BELT (EACH)]
4,6,[CAM SHAFT]
5,7,"[CONSOLE LID, CARPET]"
6,8,[RACK AND PINION MANUAL]
7,9,[COIL]
8,10,[WATER PUMP]
9,11,"[THROTTLE BODY, COIL, STARTER]"


In [6]:
# Sorting each list of part names and converting to a tuple
grouped_df['SortedPartNames'] = grouped_df['PartNames'].apply(lambda x: tuple(sorted(x)))

# Group by the sorted part names and count the occurrences
count_df = grouped_df.groupby('SortedPartNames').size().reset_index(name='Count')

# Renaming the columns for clarity
count_df.columns = ['TransactionLists', 'Count']

In [7]:
filtered_count_df = count_df[count_df['TransactionLists'].apply(lambda x: len(x) >= 2)]

In [8]:
filtered_count_df.nlargest(500, 'Count').to_csv('POS_Transactions_Count.csv', index=False)

This section performs the analysis of how often each item appears in a transaction with each other item.

In [9]:
from itertools import combinations
import pandas as pd
transactions = grouped_df["PartNames"]
unique_parts = set(part for sublist in transactions for part in sublist)

combs = list(combinations(unique_parts, 2))

comb_counts = {comb: 0 for comb in combs}

for transaction in transactions:
    for comb in combinations(transaction, 2):
        if comb in comb_counts:
            comb_counts[comb] += 1
        elif (comb[1], comb[0]) in comb_counts:
            comb_counts[(comb[1], comb[0])] += 1
            
comb_df = pd.DataFrame(list(comb_counts.items()), columns=['Combination', 'Count'])


In [10]:
comb_df['Count'] = comb_df['Count'].astype(int) # make sure these are integers so they can be properly sorted
comb_df = comb_df.sort_values(by="Count", ascending=False).reset_index(drop=True)
comb_df.to_csv('POS_Transactions_Combinations.csv', index=False)

Create the second data frame from the combination dataframe. Step 1: Search through all transactions and create a items sold dataframe of the form (PartName, Qty, TotalPrice). Step 2: Loop through the combo data frame and create copies of each combination as A/B and one as B/A. Step 3: Match counts of the items sold dataframe to the combo dataframe to produce a ratio for each A/B and B/A record.

In [11]:
# Step 1: Search through all transactions and create a items sold dataframe of the form (PartName, Qty, TotalPrice).
part_count_df = POS_transactions()
part_count_df.head()

,PartRno,PartId,PartName,LineItemID,TransactionID,Seq,PartRno,PartPriceRno,Qty,PartMeasureQty,UnitPrice,ExtPrice,CreDtTm
0,202,P70831298,TAIL GATE,3,2,1,202,202.0,1,1.000,29.9900,29.9900,2021-03-19 09:51:01.643
1,122,I70831252,SEAT BUCKET MANUAL,4,3,1,122,122.0,2,1.000,15.9900,31.9800,2021-03-19 10:02:09.660
2,75,E70831154,COIL,5,4,1,75,75.0,1,1.000,4.9900,4.9900,2021-03-19 10:07:59.860
3,125,I70831255,SEAT BELT (EACH),6,5,1,125,125.0,2,1.000,5.9900,11.9800,2021-03-19 11:04:12.900
4,10,A70831176,CAM SHAFT,7,6,1,10,10.0,1,1.000,15.9900,15.9900,2021-03-19 11:05:31.170


In [12]:
part_count_df = part_count_df[["PartName", "Qty", "UnitPrice"]]
part_count_df['TotalPrice'] = part_count_df['Qty'] * part_count_df['UnitPrice']

# Count the number of times each PartName appears
part_count_series = part_count_df['PartName'].value_counts().reset_index()

# Rename the columns to make it clear
part_count_series.columns = ['PartName', 'PartCount']

# Merge this count back with the original part_count_df if needed
part_count_df = pd.merge(part_count_df, part_count_series, on='PartName', how='left')

In [13]:
part_count_df.head()

,PartName,Qty,UnitPrice,TotalPrice,PartCount
0,TAIL GATE,1,29.9900,29.9900,197
1,SEAT BUCKET MANUAL,2,15.9900,31.9800,873
2,COIL,1,4.9900,4.9900,588
3,SEAT BELT (EACH),2,5.9900,11.9800,177
4,CAM SHAFT,1,15.9900,15.9900,87


In [14]:
grouped_df = part_count_df.groupby('PartName').agg({'TotalPrice': 'sum'}).reset_index()
grouped_df['PartCount'] = part_count_df['PartCount']

In [15]:
grouped_df.head()

,PartName,TotalPrice,PartCount
0,A FRAME,149.9400,197
1,A/C COMPRESSOR,28717.2100,873
2,A/C CONDENSER,13661.7500,588
3,A/C EVAPORATOR,286.7900,177
4,A/C REGULATOR,7.9900,87


In [16]:
# Step 2: Loop through the combo data frame and create copies of each combination as A/B and one as B/A.
def create_combination_pairs(row):
    part1, part2 = row['Combination']
    return pd.DataFrame({
        'Part1': [part1, part2],
        'Part2': [part2, part1],
        'Count': [row['Count'], row['Count']]
    })

# Apply the function to each row and concatenate the results
expanded_df = pd.concat(comb_df.apply(create_combination_pairs, axis=1).tolist()).reset_index(drop=True)


In [17]:
expanded_df.shape

(187922, 3)

In [25]:
# Step 3: Match counts of the items sold dataframe to the combo dataframe to produce a ratio for each A/B and B/A record.
# First merge on Part1
merged_df_part1 = pd.merge(expanded_df, grouped_df, left_on='Part1', right_on='PartName', how='left')
merged_df_part1 = merged_df_part1.rename(columns={'Qty': 'Qty_Part1', 'TotalPrice': 'TotalPrice_Part1'})

# Second merge on Part2
final_merged_df = pd.merge(merged_df_part1, grouped_df, left_on='Part2', right_on='PartName', how='left')
final_merged_df = final_merged_df.rename(columns={'Qty': 'Qty_Part2', 'TotalPrice': 'TotalPrice_Part2'})

# Remove extra columns
final_merged_df = final_merged_df.drop(columns=['PartName_x', 'PartName_y', 'PartCount_y'], axis='columns')

# Include part ratio

# Include Average part price
final_merged_df.head(25)


,Part1,Part2,Count,TotalPrice_Part1,PartCount_x,TotalPrice_Part2
0,TIRE 13-16,WHEEL STEEL,462,73497.1300,86,7007.3800
1,WHEEL STEEL,TIRE 13-16,462,7007.3800,157,73497.1300
2,WINDOW REGULATOR,WINDOW MOTOR,416,8204.7200,1590,8200.4100
3,WINDOW MOTOR,WINDOW REGULATOR,416,8200.4100,718,8204.7200
4,HOOD,FENDER CAR,397,46661.8400,505,50619.3500
5,FENDER CAR,HOOD,397,50619.3500,349,46661.8400
6,BUMPER STEEL,HEADLAMP COMPOSITE,393,68959.9400,190,60749.9900
7,HEADLAMP COMPOSITE,BUMPER STEEL,393,60749.9900,227,68959.9400
8,FENDER CAR,HEADLAMP COMPOSITE,384,50619.3500,349,60749.9900
9,HEADLAMP COMPOSITE,FENDER CAR,384,60749.9900,227,50619.3500
